In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
import re
import requests
from glob import glob

from multiprocessing.pool import ThreadPool

In [2]:
# def df(wow):
#     yeji = DataFrame(columns = pd.Index(['movie_id'
#                                          , 'user_id'
#                                          , 'nick'
#                                          , 'score'
#                                          , 'grade'
#                                          , 'time']))
#     return yeji

In [3]:
def movie_grade(movie_id, re_grade_count):
    url = f'https://movie.daum.net/moviedb/grade?movieId={movie_id}'
    urls = f'&type=netizen&page=1'
    double = url + urls
    daum_url = requests.get(double)

    soup = BeautifulSoup(daum_url.text, 'html.parser')
    
    wow = []
    
    for page in range(1, int(int(re_grade_count)/10+2)):
        url = 'https://movie.daum.net/moviedb/grade?movieId='
        urls = '&type=netizen&page='
        daum_url = requests.get(f'{url}' f'{movie_id}' f'{urls}' f'{page}')

        soup = BeautifulSoup(daum_url.text, 'html.parser')

        list_ = soup.select('#mArticle > div.detail_movie.detail_rating > div.movie_detail > div.main_detail > ul > li')
        nick = []

        if len(list_) == 1 :
            grade_name = soup.select('li.fst > div > a.link_review')[0].text # .translate({ord(c):None for c in '\n\t\r'})
            re_grade_name = grade_name.translate({ord(c):None for c in '\n\t\r'})
            grade_grade = soup.select('li.fst > div > div.raking_grade > em')[0].text
            grade_story = soup.select('li.fst > div > p')[0].text.strip()
            pattern = re.compile(r'\s+')
            re_grade_story = re.sub(pattern, '', grade_story)
            grade_time = soup.select('li.fst > div > div.append_review > span.info_append')[0].text
            re_grade_time = grade_time.translate({ord(c):None for c in '\n\t\r'})

            for i in list_:
                a = i.find('a')['href']
                nick.append(a.split('=')[1])
                
            wow.append([movie_id, nick[0], re_grade_name, grade_grade, re_grade_story, re_grade_time])
            
        if len(list_) > 1 :
            grade_name = soup.select('li.fst > div > a.link_review')[0].text.translate({ord(c):None for c in '\n\t\r'})
            re_grade_name = grade_name.translate({ord(c):None for c in '\n\t\r'})
            grade_grade = soup.select('li.fst > div > div.raking_grade > em')[0].text
            grade_story = soup.select('li.fst > div > p')[0].text.strip()
            pattern = re.compile(r'\s+')
            re_grade_story = re.sub(pattern, ' ', grade_story)
            grade_time = soup.select('li.fst > div > div.append_review > span.info_append')[0].text
            re_grade_time = grade_time.translate({ord(c):None for c in '\n\t\r'})

            for i in list_:
                a = i.find('a')['href']
                nick.append(a.split('=')[1])
    
            wow.append([movie_id, nick[0], re_grade_name, grade_grade, re_grade_story, re_grade_time])
            remainder = np.arange(2,len(list_)+1)        
            for i in remainder:
                grade_names = soup.select(f'ul > li:nth-of-type({i}) > div > a.link_review')[0].text
                re_grade_names = grade_names.translate({ord(c):None for c in '\n\t\r'})
                grade_grades = soup.select(f'ul > li:nth-of-type({i}) > div > div.raking_grade > em')[0].text
                grade_storys = soup.select(f'ul > li:nth-of-type({i}) > div > p.desc_review')[0].text.strip()
                pattern = re.compile(r'\s+')
                re_grade_storys = re.sub(pattern, ' ', grade_storys)
                grade_time = soup.select(f'ul > li:nth-of-type({i}) > div > div.append_review > span.info_append')
                if grade_time:
                    grade_times = soup.select(f'ul > li:nth-of-type({i}) > div > div.append_review > span.info_append')[0].text
                    re_grade_times = grade_times.translate({ord(c):None for c in '\n\t\r'})
                else:
                    continue
                nick_ = soup.select(f'ul > li:nth-of-type({i}) > div > a.link_review')
                nicks = str(nick_)
                nicks = nicks.split('=')[3]

                wow.append([movie_id, nicks, re_grade_names, grade_grades, re_grade_storys, re_grade_times])                

    yeji = DataFrame(wow,columns = pd.Index(['movie_id', 'user_id', 'nick', 'score', 'grade', 'time']))    
    return yeji


In [4]:
# movie_grade(1,105)

In [5]:
# movie_concat = pd.read_csv('./review.csv').sort_values(by=['movie_id'])
# movie_concat[movie_concat['movie_id']==1]['review_count']

In [6]:
movie_concat = pd.read_csv('./review.csv', index_col=[0]).sort_values(by=['movie_id'])
movie_concat['movie_id'][0]

1

In [7]:
def df_movie(movie_start,movie_end,movie_concat = pd.read_csv('./review.csv',index_col=[0]).sort_values(by=['movie_id'])):
    df = DataFrame(columns = pd.Index(['movie_id'
                                       , 'user_id'
                                       , 'nick'
                                       , 'score'
                                       , 'grade'
                                       , 'time']))
    for i in range(movie_start,movie_end):
        df = pd.concat([df
                        ,movie_grade(movie_concat['movie_id'][i]
                                     ,movie_concat[movie_concat['movie_id']==movie_concat['movie_id'][i]]['review_count'])])
    return df

In [8]:
df_movie(0,5)

,movie_id,user_id,nick,score,grade,time
0,1,N29meEk,도투,10,봉준호다운 작품. 사회비판과 유머가 버무려진 블랙코미디,"2020.04.12, 20:53"
1,1,RFVZNzY,박준엽,7,봉준호 감독의 특유의 블랙 코미디의 진수,"2020.03.28, 15:42"
2,1,N200Ug,MinhoCho,10,조아라 사랑해,"2020.03.24, 22:49"
3,1,QmwwYXI,최원희,9,,"2020.03.18, 09:09"
4,1,TXZsMQ,하이에너,8,강아지 던지는것 보신탕 그런것으로 이상하다고 하는 사람들 이 영화가 몇년도 작품인줄...,"2020.02.24, 01:12"
...,...,...,...,...,...,...
41,5,M1pTZVE,김부,9,기대보다 재미있었던영화!,"2008.06.01, 10:07"
42,5,UjV3VQ,nicky,10,정말 좋은 훈훈한 영화네요.. 삶과 죽음에 대한 성찰.. 좋았습니다.,"2008.06.02, 12:43"
43,5,M3hFVVc,Steel,9,이런 영화가 대한민국에 또다시 나올 수 있을까요??,"2008.05.23, 20:05"
44,5,N0hDYWs,윈저,9,좋은 영화,"2007.11.28, 21:05"


In [9]:
# review 폴더 생성
# range(pre_count, movie_len,(범위지정))
def save_review(movie_len): 
    file_list = glob('./reivew/*')
    pre_count = int(file_list[-1][-10:-4] if file_list else 0)
    try:
        if pre_count < movie_len:
            count_list = list(range(pre_count,movie_len,5))+[movie_len]
            for i in range(len(count_list)):
    #             print(count_list[i], count_list[i+1])
                df_movie(count_list[i],count_list[i+1]).to_csv('./review/%06d.csv'%(count_list[i+1])
                                                           ,encoding='utf-8'
                                                           ,index=False
                                                           )

                pre_count = count_list[i+1]
    except IndexError:
        pass

In [10]:
save_review(10)

IndexError: list index out of range

In [40]:
# [[1],[2],[3]]

In [41]:
# save_review(movie_id개수 정하기)

with ThreadPool(5) as tp:
    yeji_list = tp.map(
          save_review
        , range(10)
    )


IndexError: list index out of range

In [13]:
pd.read_csv('./review/000006.csv')

,movie_id,user_id,nick,score,grade,time
0,6,QkJ1Tw,rkdmfdl,8,秀作 수작 !!!,"2020.01.25, 14:01"
1,6,MmZIMWY,Orange-Sunshine,7,조상현 명창의 흐름과 결을 따라 함께하는 뮤지컬 또는 뮤비 춘향뎐.,"2019.12.14, 17:20"
2,6,OVJwcW4,Samuel Lee,10,아침 티비 켜자 눈에 들어온 작품! 옛날엔 그냥 즐겨 신나게 보았지만....임권택 ...,"2019.09.15, 08:21"
3,6,NEpQdHA,우산,9,한국적 리듬과 한국적 형식. 영화매체의 끝 없는 탐구.,"2019.08.18, 02:46"
4,6,RjVKVw,등록할 수 없는 닉네임입니다,1,NaN,"2019.01.28, 01:29"
5,6,SVBRRQ,이로사,10,NaN,"2019.01.20, 09:03"
6,6,RmF1eg,물에빠진물고기,5,춘향전도 이젠 시대에 맞지 않은 판소리가 된 것 같다. 요즘 여성주의 관점에서 보면...,"2018.10.19, 04:18"
7,6,RlFKUQ,국어사랑,10,NaN,"2018.09.20, 21:23"
8,6,OFFBM0g,뭘봐,9,사실 춘향전 원작자체가 19금,"2018.04.29, 23:54"
9,6,MVR3ZEY,피카피카피카츄,9,너무나도 훌륭한 작품이나 12세 관람가는 진짜 아닌 것 같습니다. 19세용이네요ㅎㄷㄷ,"2018.04.27, 14:35"


In [29]:
pd.read_csv('./review/000007.csv')

,movie_id,user_id,nick,score,grade,time
0,6,QkJ1Tw,rkdmfdl,8,秀作 수작 !!!,"2020.01.25, 14:01"
1,6,MmZIMWY,Orange-Sunshine,7,조상현 명창의 흐름과 결을 따라 함께하는 뮤지컬 또는 뮤비 춘향뎐.,"2019.12.14, 17:20"
2,6,OVJwcW4,Samuel Lee,10,아침 티비 켜자 눈에 들어온 작품! 옛날엔 그냥 즐겨 신나게 보았지만....임권택 ...,"2019.09.15, 08:21"
3,6,NEpQdHA,우산,9,한국적 리듬과 한국적 형식. 영화매체의 끝 없는 탐구.,"2019.08.18, 02:46"
4,6,RjVKVw,등록할 수 없는 닉네임입니다,1,NaN,"2019.01.28, 01:29"
...,...,...,...,...,...,...
107,7,NThpbHo,KAYOBOX,10,여운이 남는 영화...,"2008.01.31, 03:09"
108,7,NWVHZWs,해리보토,5,흠,"2007.09.25, 22:55"
109,7,MmRMeHI,왕심릉 DaDaDa,9,웃고 싶다면 이영화를 추천,"2006.03.27, 23:02"
110,7,T0pwUA,이은미,10,NaN,"2006.02.25, 00:00"


In [74]:
# movie_concat = pd.read_csv('./review.csv')

In [75]:
# movie_concat[movie_concat['movie_id']<=10]['review_count'].sum()